In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Data perprocessing expalined in Ensemble1.ipynb file

In [ ]:
policy_df =pd.read_csv('policies.csv')
driver_df =pd.read_csv('drivers.csv')
vehicle_df =pd.read_csv('vehicles.csv')

In [ ]:
policy_df = policy_df[policy_df['split'] == "Train"]

In [ ]:
policy_df.drop(['Unnamed: 0','Agent_cd','zip','state_id', 'county_name','CAT_zone', 'split'], axis = 1, inplace = True)
policy_df.info()

In [ ]:
policy_df.drop(['num_loaned_veh', 'num_owned_veh', 'num_leased_veh'], axis = 1, inplace = True)
policy_df.info()

In [ ]:
policy_df['Quote_dt'] = pd.to_datetime(policy_df['Quote_dt'])
policy_df['Quote_month'] = policy_df['Quote_dt'].dt.month
policy_df['Quote_quarter'] = policy_df['Quote_dt'].dt.quarter
policy_df['Quote_year'] = policy_df['Quote_dt'].dt.year

In [ ]:
policy_df.drop(['Quote_dt'], axis = 1, inplace = True)
policy_df.info()

In [ ]:
policy_df['quote'] = policy_df['quoted_amt'].str.replace(',','')
policy_df['quote'] = policy_df['quote'].str.replace('$','')
policy_df['quote'] = pd.to_numeric(policy_df['quote'], errors='coerce')
policy_df['quote'].dtype

In [ ]:
policy_df['quote'].describe()

changequote

In [ ]:
policy_df['quote_cats'] = pd.qcut(policy_df['quote'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

In [ ]:
policy_df.drop(['quote'], axis = 1, inplace = True)
policy_df.info()

In [ ]:
policy_df['primary_parking'].value_counts()

In [ ]:
policy_df['Cov_package_type'].value_counts()

In [ ]:
policy_df['Prior_carrier_grp'].value_counts()

In [ ]:
policy_df['credit_score'] = pd.to_numeric(policy_df['credit_score'])
score_ranges = [0, 579, 669, 739, 799, 850]
score_categories = ['Very Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
policy_df['score_category'] = pd.cut(policy_df['credit_score'],
                                     bins=score_ranges, labels=score_categories)

In [ ]:
policy_df.head()

In [ ]:
policy_df.drop(['quoted_amt', 'credit_score' ], axis = 1, inplace = True)

In [ ]:
policy_df.head()

In [ ]:
policy_df = pd.get_dummies(policy_df, columns = ['discount','Home_policy_ind','Prior_carrier_grp','Cov_package_type',
                                                 'Quote_month', 'quote_cats', 'Quote_quarter', 'Quote_year', 'score_category',
                                                 'primary_parking'])


In [ ]:
#policy_df..columns()
policy_df.head()
#policy_df.describe()


In [ ]:
np.shape(policy_df)
#Cleaned Policy Datset

Drivers Dataset

In [ ]:
driver_df.head()

In [ ]:
driver_df.describe()

In [ ]:
driver_df.columns

In [ ]:
driver_df.info()

In [ ]:
driver_df[driver_df['age']>100].describe()

In [ ]:
driver_df = driver_df.drop(driver_df[driver_df['age'] > 100].index)

In [ ]:
driver_df['Age_group'] = pd.cut(driver_df['age'], 
                         [0, 19, 34, 44, 64, 84], 
                         labels=['Teenagers', 'Young adults', 'Middle-aged adults','Old Adults', 'Older adults'])



In [ ]:
driver_df.drop(['age'], axis = 1, inplace = True)

In [ ]:
driver_df = pd.get_dummies(driver_df, columns = ['gender', 'living_status', 'Age_group' ,'high_education_ind'])
driver_df.head()

In [ ]:
bool_cols1 = [col for col in driver_df 
              if np.isin(driver_df[col].dropna().unique(), [0, 1]).all()]
bool_cols1
safty_rating_mean = driver_df['safty_rating']
if safty_rating_mean.isnull().values.any():
    mean_rating = driver_df['safty_rating'].mean()
    safty_rating_mean.fillna(mean_rating, inplace=True)


d1 = driver_df.groupby('policy_id', as_index=False)['safty_rating'].mean().rename(columns={'safty_rating': 'safty_rating_mean'})

driver_df = pd.merge(driver_df,d1, on='policy_id')

In [ ]:
driver_df.drop(['safty_rating'], axis = 1, inplace = True)

In [ ]:
bins = [-float('inf'), 20, 40, 60, 80, float('inf')]

labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

driver_df['safety_mean_cat'] = pd.cut(driver_df['safty_rating_mean'], bins=bins, labels=labels)

# Check the distribution of the new column
driver_df['safety_mean_cat'].value_counts()

In [ ]:
driver_df = pd.get_dummies(driver_df, columns = ['safety_mean_cat' ])

In [ ]:
driver_df.drop(['safty_rating_mean'], axis = 1, inplace = True)

Vehicles Dataset

In [ ]:
vehicle_df.head()

In [ ]:
vehicle_df.describe()

In [ ]:
vehicle_df.info()

In [ ]:
vehicle_df.drop(['make_model'], axis = 1, inplace = True)
vehicle_df.info()

In [ ]:
vehicle_df = pd.get_dummies(vehicle_df, columns = ['ownership_type', 'color' ])
vehicle_df.head()

In [ ]:
bool_cols1 = [col for col in vehicle_df 
              if np.isin(vehicle_df[col].dropna().unique(), [0, 1]).all()]
bool_cols1
age_mean = vehicle_df['age']
if age_mean.isnull().values.any():
    mean_age = vehicle_df['age'].mean()
    age_mean.fillna(mean_age, inplace=True)


d1 = vehicle_df.groupby('policy_id', as_index=False)['age'].mean().rename(columns={'age': 'age_mean'})

vehicle_df = pd.merge(vehicle_df,d1, on='policy_id')


In [ ]:
vehicle_df['age_mean'].describe()


In [ ]:
vehicle_df['age_mean'].fillna(vehicle_df['age_mean'].mean(), inplace=True)

In [ ]:
vehicle_df.columns

In [ ]:
na_values = vehicle_df['age_mean'].isna().sum()
print("Number of NA values in age_class column:", na_values)


In [ ]:
age_bins = [0, 4, 8, 12, 16]

age_labels = ['age_0_4', 'age_4_8', 'age_8_12', 'age_12_16']

vehicle_df['age_class'] = pd.cut(vehicle_df['age_mean'], bins=age_bins, labels=age_labels)

print(vehicle_df['age_class'].value_counts())


In [ ]:
na_values = vehicle_df['age_class'].isna().sum()

print("Number of NA values in age_class column:", na_values)


In [ ]:
vehicle_df['age_class'].fillna('age_4_8', inplace=True)


In [ ]:
na_valuesa = vehicle_df['age_class'].isna().sum()

print("Number of NA values in age_class column:", na_valuesa)


In [ ]:
vehicle_df = pd.get_dummies(vehicle_df, columns = ['age_class' ])


In [ ]:
vehicle_df.drop(['age_mean'], axis = 1)

In [ ]:
policydriver = pd.merge(left = policy_df, right = driver_df, how = 'inner', on = 'policy_id')

In [ ]:
pdv_df= pd.merge(left = policydriver, right = vehicle_df, how = 'inner', on = 'policy_id')

In [ ]:
pdv_df.head()

In [ ]:
pdv_df.describe()

In [ ]:
pdv_df.info()

In [ ]:
pdv_df.columns

In [ ]:
np.shape(pdv_df)

In [ ]:
pdv_df.to_csv('pvd_XGB_RF.csv', index=False)

In [ ]:
Dataset=pdv_df

In [ ]:
issued = Dataset[Dataset['convert_ind']==1]
ntissued = Dataset[Dataset['convert_ind']==0]

In [ ]:
#Method1 
np.random.seed(123)
issued_sample1 = issued.sample(4118)
ntissued_sample1 = ntissued.sample(4118)
Dataset1 = pd.concat([issued_sample1,ntissued_sample1],axis=0,ignore_index=True)
Dataset1 = Dataset1.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
#Method2
issued = Dataset[Dataset['convert_ind']==1]
ntissued = Dataset[Dataset['convert_ind']==0]
np.random.seed(123)
issued_sample2 = issued.sample(np.shape(issued)[0])
ntissued_sample2 = ntissued.sample(np.shape(ntissued)[0])
 
Dataset2 = pd.concat([issued_sample2,ntissued_sample2],axis=0,ignore_index=True)
Dataset2 = Dataset2.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
np.shape(issued)[0]

In [ ]:
Y1 =Dataset1['convert_ind']
X1 =Dataset1.drop(['convert_ind','policy_id'], axis = 1)
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.3, random_state=42)
X_train1, X_val1, Y_train1, Y_val1 = train_test_split(X_train1, Y_train1, test_size=0.15, random_state=42)
X_train1 = X_train1.astype('float32')
Y_train1 = Y_train1.astype('float32')
X_val1 = X_val1.astype('float32')
Y_val1 = Y_val1.astype('float32')
X_test1 = X_test1.astype('float32')
Y_test1 = Y_test1.astype('float32')
print('Training set shape:', X_train1.shape, Y_train1.shape)
print('Validation set shape:', X_val1.shape, Y_val1.shape)
print('Testing set shape:', X_test1.shape, Y_test1.shape)

In [ ]:
Y2 =Dataset2['convert_ind']
X2 =Dataset2.drop(['convert_ind','policy_id'], axis = 1)
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, test_size=0.3, random_state=42)
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train2, Y_train2, test_size=0.15, random_state=42)
X_train2 = X_train2.astype('float32')
Y_train2 = Y_train2.astype('float32')
X_val2 = X_val2.astype('float32')
Y_val2 = Y_val2.astype('float32')
X_test2 = X_val2.astype('float32')
Y_test2 = Y_val2.astype('float32')
print('Training set shape:', X_train2.shape, Y_train2.shape)
print('Validation set shape:', X_val2.shape, Y_val2.shape)
print('Testing set shape:', X_test2.shape, Y_test2.shape)

In [ ]:
print(X_train1.isna().sum())
print(X_train1.isna().any().any())
print(X_train1[X_train1.isna().any(axis=1)])


In [ ]:
cols_with_missing1 = X_train1.columns[X_train1.isna().any()].tolist()
cols_with_missing1

In [ ]:
print(X_train2.isna().sum())
print(X_train2.isna().any().any())
print(X_train2[X_train2.isna().any(axis=1)])


In [ ]:
cols_with_missing2 = X_train2.columns[X_train2.isna().any()].tolist()
cols_with_missing2

## DNN

We implemented DNN and use "RandomizedSearchCV" and "GridSearchCV" for fine tuning.

Implementing RandomizedSearchCV method for DNN

In [ ]:
import numpy as np
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, LearningRateScheduler


def create_dnn_model(hidden_layers=1, neurons=128, dropout_rate=0.5, l2_lambda=0.01, use_batchnorm=False, activation="relu", optimizer ="adam"):
    dnn_model = Sequential()
    dnn_model.add(Dense(neurons, activation=activation, input_dim=X_train1.shape[1], kernel_regularizer=l2(l2_lambda)))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(dropout_rate))
    for i in range(hidden_layers):
        dnn_model.add(Dense(neurons, activation=activation, kernel_regularizer=l2(l2_lambda)))
        if use_batchnorm:
            dnn_model.add(BatchNormalization())
        dnn_model.add(Dropout(dropout_rate))
    dnn_model.add(Dense(1, activation='sigmoid'))
    dnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model


# Define the hyperparameters for RandomizedSearchCV
param_distributions = {
    "hidden_layers": range(1, 6),
    "neurons": range(5, 30),
    "dropout_rate":[0.1, 0.2, 0.3, 0.4, 0.5],
    "l2_lambda": [0.01, 0.1, 1],
    "use_batchnorm": [False, True],
    "activation": ["relu", "tanh"],
    "optimizer": ["adam", "sgd", "rmsprop"]
}

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1_score': make_scorer(f1_score)}
def lr_schedule(epoch):
    lr = 0.0001
    for i in range(1, 2):
        if epoch >= 250 * i:
            lr /= 2
    print(f'Learning rate at epoch {epoch}: {lr}')
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1)



dnn_model = KerasClassifier(build_fn=create_dnn_model, verbose=1, epochs=500)

random_search = RandomizedSearchCV(estimator=dnn_model, param_distributions=param_distributions, n_iter=100, cv=5, n_jobs=-1, verbose=2, scoring=scoring, refit='accuracy')
random_result = random_search.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300, batch_size=256, shuffle=True, callbacks=[lr_scheduler, early_stop])

best_model = random_result.best_estimator_.model
history = best_model.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300, batch_size=256, shuffle=True, callbacks=[lr_scheduler, early_stop])

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv('random_search_results.csv', index=False)

best_model = create_dnn_model(**random_search.best_params_)
best_model.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300, batch_size=256,
                         shuffle=True)

print(f"Best parameters: {random_search.best_params_}")
print(f"Best score: {random_search.best_score_}")

results_df = pd.DataFrame.from_dict(random_search.cv_results_)
results_df.to_csv('random_search_results.csv', index=False)

with open('rnd_best_parameters.txt', 'w') as f:
    f.write(str(random_search.best_params_))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


y_pred = best_model.predict(X_test1)
y_pred_DNN_binary_rnd = (y_pred > 0.5).astype(int)

# Classification report
print(classification_report(Y_test1, y_pred_DNN_binary_rnd))

# Confusion Matrix
cm = confusion_matrix(Y_test1, y_pred_DNN_binary_rnd)
print(cm)

# Save predictions and true labels to CSV files
pd.DataFrame({'y_pred': y_pred_DNN_binary_rnd.ravel()}).to_csv('ypred_DNN_rnd.csv', index=False)
pd.DataFrame({'Y_test': Y_test1}).to_csv('Y_test.csv', index=False)

# Plot train and validation loss values
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

# Plot train and validation accuracy values
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(train_acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

Implementing GridSearchCV method for DNN

In [ ]:
import numpy as np
import os
import csv
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(hidden_layers=1, neurons=8, dropout_rate=0.1, l2_lambda=0.01, use_batchnorm=False, activation='relu', optimizer='adam'):
    dnn_model = Sequential()
    dnn_model.add(Dense(neurons, activation=activation, input_dim=X_train1.shape[1], kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda)))
    if use_batchnorm:
        dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(dropout_rate))
    for i in range(hidden_layers):
        dnn_model.add(Dense(neurons, kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda)))
        if use_batchnorm:
            dnn_model.add(BatchNormalization())
        dnn_model.add(Dropout(dropout_rate))
    dnn_model.add(Dense(1, activation='sigmoid'))

    dnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, callbacks=[lr_scheduler, early_stop])

param_grid = {
    "hidden_layers": [1, 2, 3, 4],
    "neurons": [8, 16, 32],
    "dropout_rate": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    "l2_lambda": [0.01, 0.1],
    "use_batchnorm": [False, True],
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'tanh', 'sigmoid'],
}

def lr_schedule(epoch):
    lr = 0.0001
    for i in range(1, 2):
        if epoch >= 250 * i:
            lr /= 2
    print(f'Learning rate at epoch {epoch}: {lr}')
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1)



dnn_model = KerasClassifier(build_fn=create_dnn_model, verbose=1, epochs=500)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, verbose=2, n_jobs=-1)
grid_result = grid.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300, 
                       batch_size=256, shuffle=True, callbacks=[lr_scheduler, early_stop])

best_model = grid_result.best_estimator_.model
history = best_model.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300,
                          batch_size=256, shuffle=True, callbacks=[lr_scheduler, early_stop])

results_df = pd.DataFrame(grid.cv_results_)
results_df.to_csv('random_search_results.csv', index=False)

best_model = create_dnn_model(**grid.best_params_)
best_model.fit(X_train1, Y_train1, validation_data=(X_val1, Y_val1), epochs=300, batch_size=256,
                         shuffle=True)


print(f"Best parameters: {grid.best_params_}")
print(f"Best score: {grid.best_score_}")

# Save results in CSV file
results_df = pd.DataFrame.from_dict(grid.cv_results_)
results_df.to_csv('grid_search_results.csv', index=False)

# Save best parameters in txt file
with open('grid_best_parameters.txt', 'w') as f:
    f.write(str(grid.best_params_))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


y_pred = best_model.predict(X_test1)
y_pred_DNN_binary_grid = (y_pred > 0.5).astype(int)

# Classification report
print(classification_report(Y_test1, y_pred_DNN_binary_grid))

# Confusion Matrix
cm = confusion_matrix(Y_test1, y_pred_DNN_binary_grid)
print(cm)

# Save predictions and true labels to CSV files
pd.DataFrame({'y_pred': y_pred_DNN_binary_grid.ravel()}).to_csv('ypred_DNN_grid.csv', index=False)
#pd.DataFrame({'Y_test': Y_test}).to_csv('Y_test.csv', index=False)

# Plot train and validation loss values
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

# Plot train and validation accuracy values
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(train_acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
#pip install xgboost
#pip install scipy

# GridSearchCV for XGB

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

param_grid = {
    'max_depth': [10, 20],
    'learning_rate': [0.001, 0.05],
    'n_estimators': [ 1000,2000,5000],
    'min_child_weight': [ 20,10],
    'subsample': [0.5,0.8],
    'colsample_bytree': [1.0, 0.8,0.1],
    'gamma': [ 0.2,0.5],
    'reg_alpha': [ 0.1,0.5],
    'reg_lambda': [ 2,5],
}

grid_search_XGB = GridSearchCV(estimator=xgb_model, param_grid=param_grid, n_jobs=-1, verbose=1,cv=2)
grid_search_XGB.fit(X_train1, Y_train1)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Best hyperparameters: ", grid_search_XGB.best_params_)
print("Best score: ", grid_search_XGB.best_score_)

best_model = grid_search_XGB.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test1)


# Convert y_pred from probabilities to binary predictions
y_pred_XGB_binary_grid = (y_pred > 0.5).astype(int)

# Print confusion matrix
print("Confusion matrix:")
print(confusion_matrix(Y_test1, y_pred_XGB_binary_grid))

# Print classification report
print("Classification report:")
print(classification_report(Y_test1, y_pred_XGB_binary_grid))

# Save the XGB predictions to a CSV file
y_pred = pd.DataFrame(y_pred_XGB_binary_grid)
y_pred.to_csv('y_pred_XGB_binary_grid.csv', index=False)


# RandomizedSearchCV for XGB

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBClassifier
from scipy.stats import uniform, randint

# Define the model
xgb_model = XGBClassifier()

# Define hyperparameters and their values for tuning
param_dist = {
    'max_depth': randint(1, 30),
    'learning_rate': uniform(0.001, 0.1),
    'n_estimators': randint(6998, 7000),
    'min_child_weight': randint(1, 30),
    'subsample': uniform(0.1, 0.9),
    'colsample_bytree': uniform(0.1, 0.9),
    'gamma': uniform(0, 1),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(1, 10),
}

# Use RandomizedSearchCV to find the best hyperparameters
random_search_XGB = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_jobs=-1, verbose=2, n_iter=100, cv=5)
random_search_XGB.fit(X_train1, Y_train1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("Best hyperparameters: ", random_search_XGB.best_params_)
print("Best score: ", random_search_XGB.best_score_)

best_model = random_search_XGB.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test1)


# Convert y_pred from probabilities to binary predictions
y_pred_XGB_binary_rnd = (y_pred > 0.5).astype(int)

# Print confusion matrix
print("Confusion matrix:")
print(confusion_matrix(Y_test1, y_pred_XGB_binary_rnd))

# Print classification report
print("Classification report:")
print(classification_report(Y_test1, y_pred_XGB_binary_rnd))

# Save the XGB predictions to a CSV file
y_pred = pd.DataFrame(y_pred_XGB_binary_rnd)
y_pred.to_csv('y_pred_XGB_binary_rnd.csv', index=False)


## Ranndom Forest, RF

# RandomizedSearchCV for RF

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint
from keras.callbacks import EarlyStopping

param_distributions = {
    'n_estimators': range(100, 3000),
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

rf = RandomForestClassifier()

random_search_RF = RandomizedSearchCV(rf, param_distributions=param_distributions, n_iter=100, cv=5)
random_search_RF.fit(X_train1, Y_train1)

In [ ]:
print("Best hyperparameters: ", random_search_RF.best_params_)
print("Best score: ", random_search_RF.best_score_)

best_model = random_search_RF.best_estimator_

y_pred = best_model.predict(X_test1)
y_pred_RF_binary_rnd = (y_pred > 0.5).astype(int)

print("Confusion matrix:")
print(confusion_matrix(Y_test1, y_pred_RF_binary_rnd))

print("Classification report:")
print(classification_report(Y_test1, y_pred_RF_binary_rnd))

y_pred = pd.DataFrame(y_pred_RF_binary_rnd)
y_pred.to_csv('y_pred_RF_binary_rnd.csv', index=False)

## GridSearchCV for RF

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint
from keras.callbacks import EarlyStopping

param_grid = {
    'n_estimators': [500100, 1000, 2000],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4,6],
    'max_features': ['sqrt', 'log2', None]
}

rf = RandomForestClassifier()

grid_search_RF = GridSearchCV(rf, param_grid=param_grid, cv=2)
grid_search_RF.fit(X_train1, Y_train1)


In [ ]:
print("Best hyperparameters: ", grid_search_RF.best_params_)
print("Best score: ", grid_search_RF.best_score_)

best_model = grid_search_RF.best_estimator_

y_pred = best_model.predict(X_test1)
y_pred_RF_binary_grid = (y_pred > 0.5).astype(int)

print("Confusion matrix:")
print(confusion_matrix(Y_test1, y_pred_RF_binary_grid))

print("Classification report:")
print(classification_report(Y_test1, y_pred_RF_binary_grid))

y_pred = pd.DataFrame(y_pred_RF_binary_grid)
y_pred.to_csv('y_pred_RF_binary_grid.csv', index=False)

## Ensembling

In [ ]:
Y_pred_DNNg= y_pred_DNN_binary_grid
Y_pred_DNNr= y_pred_DNN_binary_rnd

In [ ]:
Y_pred_XGBg= y_pred_XGB_binary_grid
Y_pred_XGBr= y_pred_XGB_binary_rnd

In [ ]:
Y_pred_RFg= y_pred_RF_binary_grid
Y_pred_RFr= y_pred_RF_binary_rnd

In [ ]:
from scipy.stats import mode
preds = np.hstack([Y_pred_DNNg.to_numpy().reshape(-1, 1), Y_pred_XGBg.to_numpy().reshape(-1, 1), Y_pred_RFg.to_numpy().reshape(-1, 1)])
ensemble_preds = mode(preds, axis=1)[0]
ensemble_preds = ensemble_preds.flatten()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

ensemble_preds = np.array(ensemble_preds)
cm = confusion_matrix(Y_test1, ensemble_preds)
report = classification_report(Y_test1, ensemble_preds)
print("Confusion matrix:")
print(cm)
print("\nClassification report:")
print(report)

In [ ]:
from scipy.stats import mode
preds = np.hstack([Y_pred_DNNr.to_numpy().reshape(-1, 1), Y_pred_XGBr.to_numpy().reshape(-1, 1), Y_pred_RFr.to_numpy().reshape(-1, 1)])
ensemble_preds = mode(preds, axis=1)[0]
ensemble_preds = ensemble_preds.flatten()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

ensemble_preds = np.array(ensemble_preds)
cm = confusion_matrix(Y_test1, ensemble_preds)
report = classification_report(Y_test1, ensemble_preds)
print("Confusion matrix:")
print(cm)
print("\nClassification report:")
print(report)